In [17]:
import pandas as pd
import numpy as np
import librosa
import librosa.feature as lib_fea
from pydub import AudioSegment

In [19]:
input_file = "Pop.mp3"
output_file = "result.wav"
  
sound = AudioSegment.from_mp3(input_file)
sound.export(output_file, format="wav")
y, sr = librosa.load(output_file)

In [11]:
feature_ext = []
for i in range(int(np.floor(len(y)/66149))):
    data = y[i*66149:(i+1)*66149]
    chroma_stft = lib_fea.chroma_stft(y=data)
    rms = lib_fea.rms(y=data)
    spectral_centroid = lib_fea.spectral_centroid(y=data)
    spectral_bandwidth = lib_fea.spectral_bandwidth(y=data)
    roll_off = lib_fea.spectral_rolloff(y=data)
    zero_crossing_rate = lib_fea.zero_crossing_rate(y=data)
    harmony = librosa.effects.harmonic(y=data)
    perceptr = librosa.effects.percussive(y=data)
    tempo = librosa.beat.tempo(y=data)
    mfcc = lib_fea.mfcc(y=data, n_mfcc=20)
    feature_per_3_sec = [chroma_stft.mean(), chroma_stft.var(), rms.mean(), rms.var(), spectral_centroid.mean(), spectral_centroid.var(), spectral_bandwidth.mean(), spectral_bandwidth.var(), roll_off.mean(), roll_off.var(), zero_crossing_rate.mean(), zero_crossing_rate.var(), harmony.mean(), harmony.var(), perceptr.mean(), perceptr.var(), tempo, mfcc[0].mean(), mfcc[0].var(), mfcc[1].mean(), mfcc[1].var(), mfcc[2].mean(), mfcc[2].var(), mfcc[3].mean(), mfcc[3].var(), mfcc[4].mean(), mfcc[4].var(), mfcc[5].mean(), mfcc[5].var(), mfcc[6].mean(), mfcc[6].var(), mfcc[7].mean(), mfcc[7].var(), mfcc[8].mean(), mfcc[8].var(), mfcc[9].mean(), mfcc[9].var(), mfcc[10].mean(), mfcc[10].var(), mfcc[11].mean(), mfcc[11].var(), mfcc[12].mean(), mfcc[12].var(), mfcc[13].mean(), mfcc[13].var(), mfcc[14].mean(), mfcc[14].var(), mfcc[15].mean(), mfcc[15].var(), mfcc[16].mean(), mfcc[16].var(), mfcc[17].mean(), mfcc[17].var(), mfcc[18].mean(), mfcc[18].var(), mfcc[19].mean(), mfcc[19].var()]

    feature_ext.append(feature_per_3_sec)

In [12]:
X_test = pd.DataFrame(feature_ext, dtype=float)

In [13]:
data_3_sec = pd.read_csv('features_3_sec.csv')
data_3_sec.drop(['filename','length'], axis=1, inplace=True)

from sklearn.utils import shuffle
data_3_sec = shuffle(data_3_sec)

In [14]:
X = data_3_sec.drop('label', axis=1)
y = data_3_sec['label']

In [15]:
from lightgbm import LGBMClassifier

lgbm_cls = LGBMClassifier(max_bin=30, learning_rate=0.1, num_iterations = 500, max_depth=20, num_leaves=40, random_state=0)
lgbm_cls.fit(X,y)

y_pred = lgbm_cls.predict(X_test)

/home/r0ckner/.local/lib/python3.8/site-packages/lightgbm/engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


In [16]:
print(max(set(list(y_pred)), key=list(y_pred).count))

pop
